### Loading the model and transcribing the sample_audio

In [9]:
# c2t model loading example
import whisper_s2t
from whisper_s2t.backends.ctranslate2.model import BEST_ASR_CONFIG


# model_kwargs = {
#     'device': 'cpu',  # Use 'cpu' if you don't have a GPU
#     'compute_type': 'int8', # Note int8 is only supported for CTranslate2 backend, for others only float16 is supported for lower precision.
#     'asr_options': BEST_ASR_CONFIG
# }



ct2_model = whisper_s2t.load_model(model_identifier="large-v2", 
                                   backend='CTranslate2', 
                                   compute_type='int8', 
                                   device='cpu',
                                   asr_options=BEST_ASR_CONFIG,
                                )

'ffmpeg' is not built with soxr resampler, using 'swr' resampler. This may degrade performance.


e:\anaconda3\envs\mals2t\lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

e:\anaconda3\envs\mals2t\lib\site-packages\whisper_s2t\speech_segmenter\frame_vad.py:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast()
e:\anaconda3\envs\mals2t\lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [ ]:

audio_path = "test2.wav"
files = [audio_path]
lang_codes = ['ml']
initial_prompts = [None]
# Direct Malayalam-to-English translation
tasks = ['translate']

ct2_out = ct2_model.transcribe_with_vad(
    files,
    lang_codes=lang_codes,
    tasks=tasks,
    initial_prompts=initial_prompts,
    batch_size=16
)

# segments = ct2_out[0]  # Get the segments from the first file's output

# cleaned_segments = [
#     seg.text for seg in segments
#     if getattr(seg, "avg_logprob", None) is not None and seg.avg_logprob > -1.0 and getattr(seg, "no_speech_prob", 1.0) < 0.5
# ]
# cleaned_transcript = " ".join(cleaned_segments)

print("c2t English translation:")
print(ct2_out[0][0])   # English text

In [ ]:
import importlib
import librosa
import soundfile as sf
import preprocess 
importlib.reload(preprocess)

output_path = "output.wav"
audio_path = "test2.wav"

audio, sr = preprocess.preprocess_audio(
    "test2.wav",
    "output.wav",
)


Loaded audio shape: (201600,), Sample Rate: 48000
Estimated speech rate: 3.81 onsets/sec
Speech rate is normal.
Original length: 201600, Processed length: 201600
RMS dB level: -16.89 dBFS


In [ ]:
import soundfile as sf
import tempfile
import numpy as np

lang_codes = ['ml']
initial_prompts = [None]
# Direct Malayalam-to-English translation
tasks = ['translate']


files = ["output.wav"]
    
ct2_out = ct2_model.transcribe_with_vad(
        files,
        lang_codes=lang_codes,
        tasks=tasks,
        initial_prompts=initial_prompts,
        batch_size=16
    )

    # cleaned_segments = [
    #     seg.text for seg in segments
    #     if getattr(seg, "avg_logprob", None) is not None and seg.avg_logprob > -1.0 and getattr(seg, "no_speech_prob", 1.0) < 0.5
    # ]
    # cleaned_transcript = " ".join(cleaned_segments)

print("c2t English translation:")
print(ct2_out)   # English text